# Scene Downstream - DINO 学習 (Google Colab)

このノートブックでは Google Colab 上で DINO 事前学習を実行します。

## 事前準備
- データ（H5 ファイル）を Google Drive の適当なフォルダに置くか、Colab にアップロードする
- リポジトリを Colab にクローンするか、Drive に置いたプロジェクトを使う

## 1. 環境セットアップ

In [ ]:
# Google Drive をマウント（データやチェックポイントを Drive に置く場合）
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# プロジェクトのパスを設定
# 方法A: リポジトリをクローンする
import os

REPO_URL = "https://github.com/YOUR_USERNAME/scene_downstream.git"  # 実際のURLに書き換え
PROJECT_ROOT = "/content/scene_downstream"

if not os.path.exists(PROJECT_ROOT):
    !git clone {REPO_URL} {PROJECT_ROOT}

os.chdir(PROJECT_ROOT)
print("Working directory:", os.getcwd())

In [ ]:
# 方法B: Drive に置いたプロジェクトを使う場合（上記のクローンをスキップしてこちらを使用）
# PROJECT_ROOT = "/content/drive/MyDrive/scene_downstream"  # 各自のパスに合わせる
# os.chdir(PROJECT_ROOT)
# print("Working directory:", os.getcwd())

In [ ]:
# 依存関係のインストール
!pip install -q torch torchvision h5py numpy PyYAML lightning lightly timm kornia

## 2. データパスの設定

In [ ]:
# データの場所を指定
# Colab のローカルにコピーした場合: /content/data/raw/scene
# Drive をマウントした場合の例: /content/drive/MyDrive/data/scene

DATA_PATH = "/content/drive/MyDrive/data/scene"  # 各自のデータパスに書き換え

# config を上書きするため、scene.yaml の data_path を更新
import yaml

scene_config_path = "configs/data/scene.yaml"
with open(scene_config_path) as f:
    scene_cfg = yaml.safe_load(f) or {}

scene_cfg["data_path"] = DATA_PATH
scene_cfg["num_workers"] = 2  # Colab では 2 が無難

with open(scene_config_path, "w") as f:
    yaml.dump(scene_cfg, f, default_flow_style=False, allow_unicode=True)

print("data_path set to:", DATA_PATH)

In [ ]:
# データフォルダの中身を確認（.h5 / .hdf5 が存在するか）
from pathlib import Path

p = Path(DATA_PATH)
if p.is_dir():
    files = list(p.rglob("*.h5")) + list(p.rglob("*.hdf5"))
    print(f"Found {len(files)} H5 file(s).")
    for f in files[:5]:
        print(" -", f.name)
    if len(files) > 5:
        print(" ...")
else:
    print("Path is not a directory or does not exist:", DATA_PATH)

## 3. 学習の実行

In [ ]:
# 学習を実行（configs/data/scene.yaml, configs/model/dino.yaml, configs/train.yaml を参照）
!python train.py

## 4. 結果の確認・保存

In [ ]:
# ログとチェックポイントの場所
!ls -la logs/

In [ ]:
# チェックポイントを Google Drive にコピー（オプション）
import shutil

DRIVE_SAVE_DIR = "/content/drive/MyDrive/scene_downstream_checkpoints"  # 任意のパス
os.makedirs(DRIVE_SAVE_DIR, exist_ok=True)
if os.path.isdir("logs"):
    dest = os.path.join(DRIVE_SAVE_DIR, "logs")
    shutil.copytree("logs", dest, dirs_exist_ok=True)
    print("Copied logs to", dest)